[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sensioai/blog/blob/master/054_pytorch_save/pytorch_save.ipynb)

# Pytorch - Guardando y Exportando Modelos

En posts anteriores hemos aprendido a utilizar la librería [Pytorch](https://pytorch.org/), viendo los [conceptos baśicos](https://sensioai.com/blog/027_pytorch_intro), cómo diseñar y entrenar [redes neuroanles](https://sensioai.com/blog/028_pytorch_nn) y a manejar [datasets](https://sensioai.com/blog/029_pytorch_datasets) de manera eficiente. Sin embargo, entrenar un modelo es solo parte del trabajo. Una vez tenemos nuestra red lista necesitamos poder guardarla en un archivo, o exportarla, para luego importarla en nuestras aplicaciones y ponerla a trabajar en un entorno de [producción](https://sensioai.com/blog/052_produccion). En este post vamos a ver las diferentes opciones que `Pytorch` nos ofrece a a la hora de exportar modelos.

## Guardando modelos

### Guardando los parámetros

La primera opción consiste en guardar sólo los parámetros de la red. Para ello, `Pytorch` nos permite guardar el `state_dict` del modelo, un `dict` de `Python` que contiene una relación directa entre todas las capas con parámetros de la red y sus valores.

En el siguiente código, utilizado ya en [este](https://sensioai.com/blog/042_cnns) post, entrenamos un modelo simple con el dataset `MNIST` y, una vez entrenado, guardamos el `state_dict` del modelo.

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, TensorDataset

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# preparamos los datos

import numpy as np


# Cargar datos desde el archivo CSV
mnist = pd.read_csv("/content/drive/MyDrive/Inteligencia Artificial 2/Datasets/Dig-MNIST.csv")

# Obtener características (píxeles) y etiquetas (números)
X = mnist.iloc[:, 1:].values  # Seleccionar todas las columnas excepto la primera como características
Y = mnist.iloc[:, 0].values   # Seleccionar la primera columna como etiquetas

# Verificar las formas de X e Y
print("Forma de X:", X.shape)
print("Forma de Y:", Y.shape)

x_2 = np.array(X)
y_2 = np.array(Y)

# normalización y split
X_train = x_2[:8000] / 255.
X_test = x_2[8000:] / 255.
y_train = y_2[:8000].astype(np.int32)
y_test = y_2[8000:].astype(np.int32)

# Suponiendo que tus datos son imágenes en escala de grises de tamaño 28x28
X_train = X_train.reshape(-1, 1, 28, 28)
X_test = X_test.reshape(-1, 1, 28, 28)
# Convertir a tensores de torch y enviar a GPU si estás utilizando CUDA
X_t = torch.from_numpy(X_train).float().cuda()
Y_t = torch.from_numpy(y_train).long().cuda()

# Crear datasets y dataloaders
train_dataset = TensorDataset(X_t, Y_t)
test_dataset = TensorDataset(torch.from_numpy(X_test).float().cuda(), torch.from_numpy(y_test).long().cuda())

batch_size = 2048

# DataLoader para el conjunto de entrenamiento
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True
)

# DataLoader para el conjunto de prueba
test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False
)

# Diccionario de DataLoaders
dataloader = {
    'train': train_loader,
    'test': test_loader
}


Forma de X: (10240, 784)
Forma de Y: (10240,)


In [4]:
# definimos el modelo

def block(c_in, c_out, k=3, p=1, s=1, pk=2, ps=2):
    return torch.nn.Sequential(
        torch.nn.Conv2d(c_in, c_out, k, padding=p, stride=s),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(pk, stride=ps)
    )

class CNN(torch.nn.Module):
  def __init__(self, n_channels=1, n_outputs=10):
    super().__init__()
    self.conv1 = block(n_channels, 64)
    self.conv2 = block(64, 128)
    self.fc = torch.nn.Linear(128*7*7, n_outputs)

  def forward(self, x):
    x = self.conv1(x)
    x = self.conv2(x)
    x = x.view(x.shape[0], -1)
    x = self.fc(x)
    return x

In [5]:
# entrenamos el modelo

def fit(model, dataloader, epochs=5):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = torch.nn.CrossEntropyLoss()
    for epoch in range(1, epochs+1):
        model.train()
        train_loss, train_acc = [], []
        bar = tqdm(dataloader['train'])
        for batch in bar:
            X, y = batch
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            y_hat = model(X)
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())
            acc = (y == torch.argmax(y_hat, axis=1)).sum().item() / len(y)
            train_acc.append(acc)
            bar.set_description(f"loss {np.mean(train_loss):.5f} acc {np.mean(train_acc):.5f}")
        bar = tqdm(dataloader['test'])
        val_loss, val_acc = [], []
        model.eval()
        with torch.no_grad():
            for batch in bar:
                X, y = batch
                X, y = X.to(device), y.to(device)
                y_hat = model(X)
                loss = criterion(y_hat, y)
                val_loss.append(loss.item())
                acc = (y == torch.argmax(y_hat, axis=1)).sum().item() / len(y)
                val_acc.append(acc)
                bar.set_description(f"val_loss {np.mean(val_loss):.5f} val_acc {np.mean(val_acc):.5f}")
        print(f"Epoch {epoch}/{epochs} loss {np.mean(train_loss):.5f} val_loss {np.mean(val_loss):.5f} acc {np.mean(train_acc):.5f} val_acc {np.mean(val_acc):.5f}")

In [6]:
model = CNN()
fit(model, dataloader)

val_loss 1.57158 val_acc 0.75553: 100%|██████████| 2/2 [00:00<00:00, 24.97it/s]


Epoch 1/5 loss 2.08610 val_loss 1.57158 acc 0.40627 val_acc 0.75553


val_loss 0.86986 val_acc 0.74276: 100%|██████████| 2/2 [00:00<00:00, 32.13it/s]


Epoch 2/5 loss 1.28931 val_loss 0.86986 acc 0.73868 val_acc 0.74276


val_loss 0.79776 val_acc 0.75382: 100%|██████████| 2/2 [00:00<00:00, 26.45it/s]


Epoch 3/5 loss 0.82580 val_loss 0.79776 acc 0.75684 val_acc 0.75382


val_loss 0.69482 val_acc 0.82365: 100%|██████████| 2/2 [00:00<00:00, 34.50it/s]


Epoch 4/5 loss 0.74933 val_loss 0.69482 acc 0.77606 val_acc 0.82365


val_loss 0.69946 val_acc 0.82764: 100%|██████████| 2/2 [00:00<00:00, 33.74it/s]

Epoch 5/5 loss 0.67304 val_loss 0.69946 acc 0.80353 val_acc 0.82764


In [7]:
# guardar modelo

PATH = './checkpoint.pt'
torch.save(model.state_dict(), PATH)

Ahora podemos cargar nuestro modelo y utilizarlo normalmente

In [8]:
# cargar modelo

model.load_state_dict(torch.load(PATH))
model.eval()

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=6272, out_features=10, bias=True)
)

In [9]:
def evaluate(model, dataloader):
    model.eval()
    model.to(device)
    bar = tqdm(dataloader['test'])
    acc = []
    with torch.no_grad():
        for batch in bar:
            X, y = batch
            X, y = X.to(device), y.to(device)
            y_hat = model(X)
            acc.append((y == torch.argmax(y_hat, axis=1)).sum().item() / len(y))
            bar.set_description(f"acc {np.mean(acc):.5f}")

In [10]:
evaluate(model, dataloader)

acc 0.82764: 100%|██████████| 2/2 [00:00<00:00, 32.32it/s]


Si bien de esta manera podemos guardar y cargar el modelo de manera eficiente, necesitamos tener un modelo instanciado para poder llamar a la función `model.load_state_dict()`. Esto significa que necesitaremos la definición de nuestro modelo allá dónde queramos importarlo (lo cual es poco flexible). Alternativamente, `Pytorch` nos permite guardar el modelo entero, y no solo el `state_dict`, de la siguiente manera.

In [11]:
torch.save(model, 'model.pt')

Y podemos cargar y evaluar nuestro modelo así

In [12]:
model = torch.load('model.pt')
model.eval()

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=6272, out_features=10, bias=True)
)

In [13]:
evaluate(model, dataloader)

acc 0.82764: 100%|██████████| 2/2 [00:00<00:00, 31.96it/s]


Si bien de esta forma no necesitamos que nuestro modelo esté instanciado, seguimos necesitando su definición. Es por este motivo que la opción anterior es la recomendada, ya que es más eficiente (sólo guardamos los pesos) y también más flexible (podemos guardar otra información además del `state_dict` de nuestro modelo). Esta opción es ideal para guardar y cargar modelos durante el entrenamiento del mismo, quizás incluso junto al estado del optimizador, de manera que podemos entrenar modelos a partir de estos `checkpoints` en lugar de empezar de cero cada vez. Otro ejemplo consistiría en guardar el `state_dict` del modelo durante el entrenamiento solo cuando mejore una métrica determinada y cargar el mejor modelo al final del entrenamiento (que no tiene porqué coincidir con el último).

> ⚡ Aprende más sobre el guardado de modelos en `Pytorch` [aquí](https://pytorch.org/tutorials/beginner/saving_loading_models.html).

In [14]:
def fit(model, dataloader, epochs=5, PATH="./checkpoint.pt"):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = torch.nn.CrossEntropyLoss()
    best_acc = 0
    for epoch in range(1, epochs+1):
        model.train()
        train_loss, train_acc = [], []
        bar = tqdm(dataloader['train'])
        for batch in bar:
            X, y = batch
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            y_hat = model(X)
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())
            acc = (y == torch.argmax(y_hat, axis=1)).sum().item() / len(y)
            train_acc.append(acc)
            bar.set_description(f"loss {np.mean(train_loss):.5f} acc {np.mean(train_acc):.5f}")
        bar = tqdm(dataloader['test'])
        val_loss, val_acc = [], []
        model.eval()
        with torch.no_grad():
            for batch in bar:
                X, y = batch
                X, y = X.to(device), y.to(device)
                y_hat = model(X)
                loss = criterion(y_hat, y)
                val_loss.append(loss.item())
                acc = (y == torch.argmax(y_hat, axis=1)).sum().item() / len(y)
                val_acc.append(acc)
                bar.set_description(f"val_loss {np.mean(val_loss):.5f} val_acc {np.mean(val_acc):.5f}")
        # guardar modelo si es el mejor
        val_acc = np.mean(val_acc)
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), PATH)
            print(f"Best model saved at epoch {epoch} with val_acc {val_acc:.5f}")
        print(f"Epoch {epoch}/{epochs} loss {np.mean(train_loss):.5f} val_loss {np.mean(val_loss):.5f} acc {np.mean(train_acc):.5f} val_acc {np.mean(val_acc):.5f}")
    # cargar el mejor modelo al final del entrenamiento
    model.load_state_dict(torch.load(PATH))

In [15]:
model = CNN()
fit(model, dataloader)

val_loss 1.67019 val_acc 0.72982: 100%|██████████| 2/2 [00:00<00:00, 30.99it/s]


Best model saved at epoch 1 with val_acc 0.72982
Epoch 1/5 loss 2.12411 val_loss 1.67019 acc 0.38167 val_acc 0.72982


val_loss 0.89624 val_acc 0.78312: 100%|██████████| 2/2 [00:00<00:00, 34.74it/s]


Best model saved at epoch 2 with val_acc 0.78312
Epoch 2/5 loss 1.40057 val_loss 0.89624 acc 0.71723 val_acc 0.78312


val_loss 0.74317 val_acc 0.78866: 100%|██████████| 2/2 [00:00<00:00, 35.09it/s]


Best model saved at epoch 3 with val_acc 0.78866
Epoch 3/5 loss 0.87548 val_loss 0.74317 acc 0.75247 val_acc 0.78866


val_loss 0.73908 val_acc 0.79508: 100%|██████████| 2/2 [00:00<00:00, 34.46it/s]


Best model saved at epoch 4 with val_acc 0.79508
Epoch 4/5 loss 0.75722 val_loss 0.73908 acc 0.77787 val_acc 0.79508


val_loss 0.69368 val_acc 0.81901: 100%|██████████| 2/2 [00:00<00:00, 36.45it/s]

Best model saved at epoch 5 with val_acc 0.81901
Epoch 5/5 loss 0.70375 val_loss 0.69368 acc 0.79948 val_acc 0.81901


In [16]:
evaluate(model, dataloader)

acc 0.81901: 100%|██████████| 2/2 [00:00<00:00, 35.28it/s]


Guardar nuestros modelos, ya sea el modelo entero o solo su `state_dict`, es la forma más directa y sencilla de guardar cualquier modelo que hagamos. Sin embargo, tiene ciertas limitaciones. Por un lado, como ya hemos visto, necesitamos la definición del modelo tanto a la hora de entrar como en producción. Esto no solo es engorroso y poco flexible, si no que sólo funcionará en entornos `Python` con `Pytorch` instalado. Si bien ésto es suficiente para, por ejemplo, poner nuestros modelos a trabajar en un servidor [Flask](https://sensioai.com/blog/052_produccion)), en muchas ocasiones necesitaremos ejecutar nuestras redes neuronales en otros entornos (smartphones, aplicaciones web, IoT, ...) en las que usaremos otros lenguajes de programación. Para ello, `Pytorch` nos permite `exportar` nuestro modelo en vez de simplemente `guradarlo`.

## Exportando modelos

### Torchscript

[Torchscript](https://pytorch.org/tutorials/beginner/Intro_to_TorchScript_tutorial.html) es una representación intermedia de un modelo de `Pytorch` que puede ejecutarse en diferentes entornos sin la necesidad de `Python`, por ejemplo en `C++`. Un modelo de `Pytorch` exportado en `torchscript` contiene los pesos de la red así como su definición (todas las operaciones que aplicaremos a un tensor desde la entrada hasta la salida). Tenemos dos maneras de exportar un modelo con `torchscript`:

- `tracing`: Dada un entrada, se genera una representación del modelo de manera dinámica registrando todas las operaciones aplicadas al tensor hasta la salida. En este caso no seremos capaces de capturar diferentes caminos en nuestra red (*control flow*). Es la alternativa más eficiente, pero menos flexible.
- `scripting`: Genera la representación intermedia de nuestra red neuronal directamente a partir del análisis de la misma, siendo capaz de capturar de manera fiel cualquier ramificación en la misma. No es tan eficiente, pero si más flexible.

In [17]:
# tracing

x = torch.rand(32, 1, 28, 28)
traced_model = torch.jit.trace(model.cpu(), x)
traced_model.save('model.zip')

In [18]:
loaded_model = torch.jit.load('model.zip')
evaluate(loaded_model, dataloader)

acc 0.81901: 100%|██████████| 2/2 [00:00<00:00, 31.52it/s]


In [19]:
# scripting

scripted_model = torch.jit.script(model.cpu())
scripted_model.save('model.zip')

In [20]:
loaded_model = torch.jit.load('model.zip')
evaluate(loaded_model, dataloader)

acc 0.81901: 100%|██████████| 2/2 [00:00<00:00, 33.18it/s]


Exportar nuestro modelo nos aporta varias ventajas:

- Ahora nuestro modelo puede ejecutarse en cualquier entorno capaz de interpretar la representación intermedia generada por `torchscript`, independientemente del hardware o software utilizado.
- Nuestro modelo contiene los pesos y la definición de las operaciones, evitando tener que guardar código extra.
- Esta representación intermedia puede ser optimizada de manera independiente, haciendo que nuestros modelos sean más rápidos.

La principal desventaja es que al estar "traduciendo" `Python` a otro lenguaje, es posible que no todas las operaciones que queramos hacer estén soportadas.

> ⚡ Aprende más sobre `Torchscript`[aquí](https://pytorch.org/tutorials/beginner/Intro_to_TorchScript_tutorial.html).

Un consejo a tener en cuenta a la hora de exportar nuestros modelos que nos puede hacer la vida más fácil cuando pongamos nuestros modelos en producción, es incluir cualquier pre- o post-procesado de datos necesarios en el mismo modelo. Durante el entrenamiento no lo hacemos por motivos de eficiencia (queremos que nuestra `GPU` entrene la red lo más rápido posible mientras la `CPU` procesa cada batch de datos de manera paralela), pero estos procesados pueden ser costosos en producción (a veces incluso imposibles de realizar) por lo que incluirlos en el modelo es generalmente una buena idea.

En este caso incluimos la normalización y preparación de los datos en un pre-procesado y calculamos una distribución de probabilidad sobre las salida con un post-procesado, que además también nos devuelve la clase con mayor probabilidad.

In [21]:
class Preprocessing(torch.nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, x):
        # esperamos un batch de imágenes sin normalizar
        # normalización
        x = (x / 255.)
        x = (x - 0.1307) / 0.3081
        # dimsensiones -> [bs, c, h, w]
        x = x.unsqueeze(1)
        # en imágenes en color, haríamos un `permute`
        return x

class Postprocessing(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.softmax = torch.nn.Softmax(dim=1)
    def forward(self, x) :
        # devolvemos distribución de probabilidad
        # y clase con mayor probabilidad
        return self.softmax(x), torch.argmax(x, dim=1)

In [22]:
final_model = torch.nn.Sequential(
    Preprocessing(),
    model.cpu(),
    Postprocessing()
)

scripted_model = torch.jit.script(final_model)
scripted_model.save('model.zip')

Ahora nuestro modelo acepta lotes de imágenes sin normalizar y con las dimensiones más comunes (alto, ancho) y devuelve una distribución de probabilidad. Es nuestro modelo quien se encarga del procesado.

In [23]:
def script_evaluate(model, dataloader):
    model = torch.jit.load(model)
    model.cuda()
    model.eval()
    bar = tqdm(dataloader['test'])
    acc = []
    with torch.no_grad():
        for batch in bar:
            X, y = batch
            # desnormalizar
            X = (X*0.3081 + 0.1307)*255
            # quitar dimensión canales
            X = X.squeeze(1)
            # el modelo pre-procesa
            y_hat, label = model(X)
            acc.append((y == label).sum().item() / len(y))
            bar.set_description(f"acc {np.mean(acc):.5f}")

In [24]:
script_evaluate("model.zip", dataloader)

acc 0.81901: 100%|██████████| 2/2 [00:00<00:00,  8.94it/s]


### ONNX

Si bien ahora nuestro modelo es capaz de ser importado y ejecutado de forma más flexible y eficiente, seguimos limitados por el hecho de necesitar `Pytorch` (o la librería `Torchscript` en `C++`) para ello. Por este motivo, `Pytorch` nos ofrece una última manera de exportar nuestros modelos a otra forma de representación intermedia conocida como [ONNX](https://onnx.ai/). Éste es un formato abierto con el espíritu de convertirse en un estándar de representación de redes neuronales. La gran mayoría de librerías y *frameworks* de `deep learning` soportan este formato, lo que implica que, por ejemplo, podemos entrenar un modelo en `Pytorch`, exportarlo en formato `ONNX` e importarlo en `Tensorflow` para ponerlo en producción (aunque `ONNX` también ofrece soluciones optimizadas para ello: `ONNX Runtime`). Así pues, exportar nuestros modelos a formato `ONNX` nos proporcionará la máxima flexibilidad, permitiéndonos, entre muchas otras cosas, ejecutar nuestras redes neuronales en entornos tales como navegadores web o IoT. Por contra, esta librería es la menos flexible en cuanto a cantidad y tipo de operaciones que podemos exportar, lo cual puede imponer unas restricciones muy grandes sobre nuestros modelos (aunque cada vez se soportan más).

In [ ]:
import os
os.kill(os.getpid(), 9)
!pip install onnx

In [ ]:
x = torch.rand(32, 1, 28, 28)
y = model.cpu()(x)

# exportamos el modelo
torch.onnx.export(model,                     # el modelo
                  x,                         # un ejemplo del input
                  "model.onnx",              # el nombre del archivo para guardar el modelo
                  export_params=True,        # guardar los pesos de la red
                  opset_version=10,          # versión de ONNX
                  do_constant_folding=True,  # optimizaciones
                  input_names = ['input'],   # nombre de los inputs
                  output_names = ['output'], # nombre de los outputs
                  dynamic_axes={'input' : {0 : 'batch_size'},    # ejes con longitud variable (para poder usar diferentes tamaños de batch)
                                'output' : {0 : 'batch_size'}})

Para poder ejecutar nuestro modelo necesitamos la librería de `ONNX Runtime` para `Python`, que puedes instalar con el comando `pip install onnxruntime`.

In [ ]:
import onnxruntime

def onnx_evaluate(model, dataloader):
    # cargarmos el modelo
    ort_session = onnxruntime.InferenceSession(model)
    bar = tqdm(dataloader['test'])
    acc = []
    with torch.no_grad():
        for batch in bar:
            X, y = batch
            X, y = X.numpy(), y.numpy()
            # generamos los inputs
            ort_inputs = {ort_session.get_inputs()[0].name: X}
            # extraemos los outputs
            ort_outs = ort_session.run(None, ort_inputs)[0]
            acc.append((y == np.argmax(ort_outs, axis=1)).mean())
            bar.set_description(f"acc {np.mean(acc):.5f}")

In [ ]:
onnx_evaluate("model.onnx", dataloader)

De nuevo, te recomiendo incluir tanto pre- como post-procesado como parte del modelo para evitar problemas más adelante.

> ⚡ Aprende más sobre `ONNX`[aquí](https://pytorch.org/tutorials/advanced/super_resolution_with_onnxruntime.html).

## Resumen

En este post hemos visto las diferentes manera que `Pytorch` nos ofrece a la hora de guardar y exportar nuestros modelos. Si en nuestro entorno de producción podemos usar `Python` e instalar `Pytorch`, entonces podemos simplemente `guardar` el `state_dict` de nuestro modelo, o incluso el modelo completo, y luego cargarlo en la aplicación. Ten en cuenta que necesitarás la definición de tu red neuronal (en nuestro ejemplo, la clase `Model`) que define las capas y operaciones que se aplican dada una entrada. Sin embargo, una opción más eficiente, consiste en `exportar` nuestro modelo con `torch.jit.trace` o `torch.jit.script` ya que luego podremos cargar nuestro modelos sin necesidad de arrastrar código y, además, podremos ejecutarlo en otros entornos como `C++`. Por último, si no puedes usar `Python` en tu aplicación, es muy posible que puedas usar `ONNX`, un formato estándar al que `Pytorch` nos permite también exportar.